In [1]:
from random import choices, choice, randint, randrange, random
from math import log10
import os
import time

In [2]:
def parse_input(filename):
    with open(filename, "r") as f:
        data = f.read()
    data = [i for i in data.split("\n") if i.strip() != ""]
    
    data = data[1:]
    client_data = [[data[i], data[i+1]] for i in range(0, len(data), 2)]
    
    for i in range(len(client_data)):
        likes, dislikes = client_data[i]
        likes = set(likes.split(" ")[1:])
        dislikes = set(dislikes.split(" ")[1:] or [])
        client_data[i] = [likes, dislikes]
    return client_data

In [3]:
def score_pizza(pizza_ingredients: set[str]):
    pizza_ingredients = set(pizza_ingredients)
    score = 0
    global client_data
    for client_info in client_data:
        client_likes, client_dislikes = client_info
        client_likes_pizza = True
        
        # all the ingredients they like are on the pizza
        for liked_ingredient in client_likes:
            if liked_ingredient not in pizza_ingredients:
                client_likes_pizza = False
                break

        # none of the ingredients they dislike are on the pizza 
        if client_likes_pizza:
            for disliked_ingredient in client_dislikes:
                if disliked_ingredient in pizza_ingredients:
                    client_likes_pizza = False
                    break
        if client_likes_pizza:
            score += 1
    return score

In [4]:
def getAllIngredients():
    global client_data
    allIngredients = set()
    for client_info in client_data:
        client_likes, client_dislikes = client_info
        allIngredients = allIngredients.union(client_likes)
        allIngredients = allIngredients.union(client_dislikes)
    return sorted(list(allIngredients))

In [5]:
def getAllDislikedIngredients():
    global client_data
    result = set() 
    for client_info in client_data:
        client_likes, client_dislikes = client_info
        result = result.union(client_dislikes)
    return result

In [6]:
def getIngredientsThatWereNeverDisliked():
    global client_data
    allIngredients = set(getAllIngredients(client_data))
    allDislikes = getAllDislikedIngredients(client_data)
    
    result = set()
    for i in allIngredients:
        if i not in allDislikes:
            result.add(i)
    
    return result

In [7]:
def generate_genome():
    'Generates random genome'
    global genomeLength
    return choices([0, 1], k=genomeLength)

def generate_population(size):
    'Generates list of random genomes of provided size'
    return [generate_genome() for _ in range(size)]

def genome_to_ingredients(genome):
    'Convert genome to what pizza ingredients they represent'
    ingredients = set()
    for idx, bit in enumerate(genome):
        if bit == 1:
            ingredients.add(allIngredients[idx])
    return ingredients

def get_genome_score(genome):
    'Get score of genome i.e. how many clients were fine with the pizza'
    ingredients = genome_to_ingredients(genome)
    return score_pizza(ingredients)

In [8]:
def selection_pair(population):
    return choices(
        population=population,
        weights=[get_genome_score(g) for g in population],
        k = 2
    )

def single_point_crossover(genome_a, genome_b):
    p = randint(1, len(genome_a)-1)
    return genome_a[0:p] + genome_b[p:], genome_b[0:p] + genome_a[p:]

def mutation(g, count = 1, probability = 0.5):
    'Flip `count` bits of genome `g` with a chance of `probability`'
    for _ in range(count):
        idx = randrange(len(g))
        if random() < probability:
            g[idx] = g[idx] ^ 1         # Bit-Flip
    return g

In [9]:
def run_evolution(population_size=10):
    global genomeLength, bestGenome, bestScore
    population = generate_population(population_size)
    population = sorted(population, key=lambda g: -get_genome_score(g))
    
    i = 0
    while True:    
        next_gen = population[:2] # Top 2 candidates
        
        for _ in range(int(len(population) / 2) - 1):
            parents = selection_pair(population)
            # Reproduce
            child_a, child_b = single_point_crossover(parents[0], parents[1])
            # Mutation
            child_a, child_b = mutation(child_a), mutation(child_b)
            # Add them to next gen
            next_gen += [child_a, child_b]
        
        population = next_gen
        population = sorted(population, key=lambda g: -get_genome_score(g))
        bestGenome = population[0]
        bestScore = get_genome_score(bestGenome)
        i += 1
        if i % 50 == 0:
            print(f"Gen: {i} | Current Best Score: {bestScore}" + " "*15, end="\r")

In [10]:
files = ['./input/' + file for file in os.listdir('./input')]
input_file = files[3]
client_data = parse_input(input_file)
allIngredients = list(getAllIngredients())
bestGenome = None
bestScore = 0
genomeLength = len(allIngredients)

print(f"File selected: {input_file}")
print(f"No. of ingredients: {len(allIngredients)}")
print(f"No. of clients: {len(client_data)}")
print(f"No. of permutations possible: 2^{len(allIngredients)} or 10^{round(log10(2)*len(allIngredients), 2)}")
print(f"Max Score Limit: {len(client_data)}")
print(time.strftime("Time started: %H:%M"))

run_evolution(10)

File selected: ./input/d_difficult.in.txt
No. of ingredients: 600
No. of clients: 9368
No. of permutations possible: 2^600 or 10^180.62
Time started: 20:39


KeyboardInterrupt: 

In [14]:
# print(bestGenome)
# print(genome_to_ingredients(bestGenome))

In [15]:
# Generation: 8674 | Max Score: 9368 | Current Best Score: 1787    
# Generation: 15847 | Max Score: 9368 | Current Best Score: 1787
# Generation: 19361 | Max Score: 9368 | Current Best Score: 1789